In [ ]:
!pip install mergekit transformers accelerate bitsandbytes huggingface_hub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token

In [ ]:
%%writefile config.yaml
models:
  - model: "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO" #OpenAI/ChatGPT-Mixtral-8x7B
    parameters:
      weight: 0.5
  - model: "VAGOsolutions/SauerkrautLM-Mixtral-8x7B-Instruct" #TheFuriousGunner/Mistral-7B-Finetuned
    parameters:
      weight: 0.25
  - model: "mistralai/Mixtral-8x7B-Instruct-v0.1"
    parameters:
      weight: 0.25

merge_method: slerp
base_model: "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
dtype: bfloat16

In [ ]:
from huggingface_hub import hf_hub_download
import json

config_path = hf_hub_download("VAGOsolutions/SauerkrautLM-Mixtral-8x7B-Instruct", "config.json")
with open(config_path) as f:
    cfg = json.load(f)
print(cfg.keys())

In [ ]:
!mkdir -p /content/drive/MyDrive/mergekit_model

In [ ]:
#!mergekit-yaml config.yaml ./merged-model
!mergekit-yaml config.yaml /content/drive/MyDrive/mergekit_model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "/content/drive/MyDrive/mergekit_model"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    local_files_only=True
)

inputs = tokenizer("Hello, what is the capital of France?", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
from transformers import AutoTokenizer

# 원래 모델 중 하나에서 tokenizer 가져오기
tokenizer_model = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

# merge된 모델 폴더로 tokenizer 저장
tokenizer.save_pretrained("./merged-model")

print("✅ Tokenizer 추가 완료")


In [ ]:
from huggingface_hub import HfApi, login

# 1️⃣ HF 로그인
login(token='HF_TOKEN')  # 이미 HF_TOKEN 있으니까 그대로 사용

# 2️⃣ 업로드 (자동으로 레포지토리 생성)
api = HfApi()
repo_id = "jsgoodlife0511/Mergekit-Mixtral-8x7B"  # 원하는 모델 이름
api.upload_folder(
    folder_path="./merged-model",  # 로컬에 merge된 모델 폴더
    repo_id=repo_id,
    repo_type="model",
    ignore_patterns=["*.lock"]
)

print(f"✅ 모델 업로드 완료! Hugging Face Hub: https://huggingface.co/{repo_id}")
